# Обучение генераитвной диалоговой модели

## Установка зависимостей

In [ ]:
!pip install evaluate datasets torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 23.5 MB/s eta 0:00:00


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import TensorDataset
import evaluate
from torch.utils.data import DataLoader
import re
from torchmetrics.text import BLEUScore
from transformers import AutoTokenizer, AutoModelForCausalLM

## Подготовка данных

In [ ]:
ds = datasets.load_dataset("igorktech/anekdots_dialogs")
ds = ds.remove_columns([
    'original', 'date', 'downvote', 'total_votes', 'upvote',
    'hash', 'alpha_frac', 'LDR', 'days_since_publication',
    'time_decay', 'LDR_time_decay'
])
ds

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/41.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100834 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['parsed', 'total_mark'],
        num_rows: 100834
    })
})

Оставляем диалоги только с двумя высказываниями

In [ ]:
filtered_ds = [i for i in ds['train'] if len(i['parsed']['turns']) == 2]
print(len(filtered_ds))

48032


Перевод в Pandas

In [ ]:
df = pd.DataFrame.from_dict(filtered_ds)
df.head()

,parsed,total_mark
0,"{'turns': [{'speaker_name': '', 'phrase': 'Вы ...",227
1,"{'turns': [{'speaker_name': '', 'phrase': 'Мор...",4
2,"{'turns': [{'speaker_name': '', 'phrase': 'Как...",-3
3,"{'turns': [{'speaker_name': '', 'phrase': 'Слы...",-5
4,"{'turns': [{'speaker_name': '', 'phrase': 'Что...",-9


Сортировка датасета по убыванию оценки

In [ ]:
top_df = df.sort_values('total_mark', ascending=False).iloc[0:10000]
top_df.head()

,parsed,total_mark
4673,"{'turns': [{'speaker_name': 'Дворецкий', 'phra...",4060
9514,"{'turns': [{'speaker_name': 'Ученики', 'phrase...",3145
28556,"{'turns': [{'speaker_name': 'Москвичка', 'phra...",3061
5395,"{'turns': [{'speaker_name': 'Водитель', 'phras...",2376
38011,"{'turns': [{'speaker_name': '', 'phrase': 'У в...",2295


## Загрузка модели

In [ ]:
model_path = 'ai-forever/rugpt3small_based_on_gpt2' # модель не диалоговая, обучалась только на задаче языкового моделирования

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

## Тестовый запуск модели

In [ ]:
text = 'Сколько человек нужно, чтобы закрутить лампочку?'
inputs = tokenizer(text, return_tensors='pt')

In [ ]:
with torch.no_grad():
    # https://huggingface.co/docs/transformers/main/en/main_classes/text_generation
    hypotheses = model.generate(
        **inputs,
        temperature=0.9,
        do_sample=True,           # sampling или greedy decoding
        top_p=0.7,
        num_return_sequences=3,
        repetition_penalty=2.5,   # https://arxiv.org/pdf/1909.05858.pdf
        max_length=32,
    )

In [ ]:
for h in hypotheses:
  print(tokenizer.decode(h, skip_special_tokens=True), end='\n\n')

Сколько человек нужно, чтобы закрутить лампочку?
1. Вкрутись в квартиру и купи ей розетку (а лучше две) 2-3

Сколько человек нужно, чтобы закрутить лампочку?
А у вас есть какие-то вопросы по этим вопросам к этому магазину или нет? Если да то

Сколько человек нужно, чтобы закрутить лампочку?
Если у Вас есть лампа в комнате - достаточно. Если нет- то можно поставить светильник и включать его



### Вывод

1. Попадаются вопросы не относящиеся к лампе вовсе
2. Генерируются длинные ответы (на чём изначально модель и обучалась)

## Подготовка датасета для обучения

In [ ]:
def tokenize_data(tokenizer, data, column_name, max_length=128):
    source_texts = data[column_name].tolist()
    tokens = tokenizer(
        source_texts,
        max_length=max_length,
        padding='max_length',
        padding_side="right",
        truncation=True,
        add_special_tokens=True,
        return_tensors='pt',
    )
    return tokens

In [ ]:
train, other = train_test_split(top_df, test_size=0.2, random_state=42)
test, valid = train_test_split(other, test_size=0.5, random_state=42)

Два варианта форматирования данных для улучшения обучения

In [ ]:
# tokenizer.add_special_tokens({'additional_special_tokens': ['<speaker1>', '<speaker2>']})
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
train['text'] = train.apply(
    lambda x: '<s> Первый: ' + x['parsed']['turns'][0]['phrase'] +
              ' Второй: ' + x['parsed']['turns'][1]['phrase'] + '</s>',
    axis=1
)

valid['text'] = valid.apply(
    lambda x: '<s> Первый: ' + x['parsed']['turns'][0]['phrase'] +
              ' Второй: ' + x['parsed']['turns'][1]['phrase'] + '</s>',
    axis=1
)

test['text'] = test.apply(
    lambda x: '<s> Первый: ' + x['parsed']['turns'][0]['phrase'] +
              ' Второй: ' + x['parsed']['turns'][1]['phrase'] + '</s>',
    axis=1
)

In [ ]:
valid['text'].iloc[1]

'<s> Первый: В Китае введена смертная казнь за хранение наркотиков! Второй: Да, но китайцы могут себе это позволить!</s>'

In [ ]:
train_tokens = tokenize_data(tokenizer, train, column_name='text')
val_tokens   = tokenize_data(tokenizer, valid, column_name='text')
test_tokens  = tokenize_data(tokenizer, test, column_name='text')

In [ ]:
train_tokens['input_ids'][125]

tensor([    1,  7200,    30,   985,  1490,   481, 49223,    35,  7409,    30,
         4777,  1139,    18,  1937, 14463,   382, 15415,    16,   417,  4765,
          306,   697,  6173,   458,   672, 32002,    16, 19083, 26350,   367,
          785, 22359,    16,   417,   773,  5755,    30,   458, 13571, 22678,
           16, 50141,  5713,     2,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
train_dataset = TensorDataset(
    train_tokens['input_ids'],
    train_tokens['attention_mask'],
)

val_dataset = TensorDataset(
    val_tokens['input_ids'],
    val_tokens['attention_mask'],
)

test_dataset = TensorDataset(
    test_tokens['input_ids'],
    test_tokens['attention_mask'],
)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=8, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=8, shuffle=False)

## Обучение диалоговой модели

In [ ]:
device = 'mps' if torch.backends.mps.is_built() else 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

Функции обучения

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_epoch(
    model,
    loader: DataLoader,
    epoch,
    num_epochs,
    optimizer,
    mode,
):
    total_loss = 0

    for x_input_ids, x_attention_mask in tqdm(loader, desc=f'{mode} epoch {epoch}/{num_epochs}...'):
        if mode == 'Training':
            optimizer.zero_grad()

        x_input_ids = x_input_ids.to(device)
        x_attention_mask = x_attention_mask.to(device)

        outputs = model(
            input_ids=x_input_ids,
            attention_mask=x_attention_mask,
            labels=x_input_ids,
            return_dict=True,
        )

        outputs_loss = outputs.loss
        total_loss += outputs_loss.item()

        if mode == 'Training':
            outputs_loss.backward()
            optimizer.step()

    loss = total_loss / len(loader)
    print(f'{mode} epoch {epoch + 1}/{num_epochs}: {mode} Loss: {loss:.4f}')

In [ ]:
def train(
    model,
    train_loader,
    val_loader,
    optimizer,
    num_epochs=5,
):
    for epoch in range(num_epochs):
        model.train()
        train_epoch(model, train_loader, epoch, num_epochs, optimizer, mode='Training')

        model.eval()
        with torch.no_grad():
            train_epoch(model, val_loader, epoch, num_epochs, optimizer, mode='Validating')

Обучение

In [ ]:
train(model, train_loader, val_loader, optimizer, num_epochs=4)

Training epoch 0/4...: 100%|██████████| 1000/1000 [04:52<00:00,  3.42it/s]


Training epoch 1/4: Training Loss: 1.1323


Validating epoch 0/4...: 100%|██████████| 125/125 [00:10<00:00, 11.51it/s]


Validating epoch 1/4: Validating Loss: 0.7688


Training epoch 1/4...: 100%|██████████| 1000/1000 [04:54<00:00,  3.40it/s]


...

Validating epoch 3/4: Validating Loss: 0.7643


Training epoch 3/4...: 100%|██████████| 1000/1000 [04:54<00:00,  3.40it/s]


Training epoch 4/4: Training Loss: 0.6957


Validating epoch 3/4...: 100%|██████████| 125/125 [00:10<00:00, 11.44it/s]

Validating epoch 4/4: Validating Loss: 0.7685


## Тестовый запуск после обучения

In [ ]:
text = '<s> Первый: Сколько человек нужно, чтобы закрутить лампочку? Второй: ' # подсказка для модели в соответствии с дообучением

# inputs = tokenizer(text, return_tensors='pt', padding_side="right",)
inputs = tokenizer(
    text,
    add_special_tokens=True,
    return_tensors='pt',
)

In [ ]:
inputs['input_ids'] = inputs['input_ids'].cuda() # перенос токенов на GPU
inputs['attention_mask'] = inputs['attention_mask'].cuda()

In [ ]:
with torch.no_grad():
    # https://huggingface.co/docs/transformers/main/en/main_classes/text_generation
    hypotheses = model.generate(
        **inputs,
        temperature=0.8,
        do_sample=True,  # sampling or greedy decoding
        # (at each decoding step selects the token with the highest prob without considering the impact on future tokens)
        top_p=0.7,
        num_return_sequences=3,
        repetition_penalty=2.5,  # https://arxiv.org/pdf/1909.05858.pdf
        max_new_tokens=128,
    )

In [ ]:
for h in hypotheses:
  print()
  print(tokenizer.decode(h, skip_special_tokens=True))


 Первый: Сколько человек нужно, чтобы закрутить лампочку? Второй:  Три.

 Первый: Сколько человек нужно, чтобы закрутить лампочку? Второй:  Один.

 Первый: Сколько человек нужно, чтобы закрутить лампочку? Второй:  Тридцать!


### Вывод

1. Модель генерирует ответ в соответствии с дообучением: "Первый: .. Второй: .."
2. Ответы стали компактнее
3. Юмора в них не прибавилось
* Возможно потому, что недостаточно данных или недостаточное дообучение. Хотя лучший лосс на второй эпохе..

## Расчёт метрики BLEU

In [ ]:
def generate(
    model,
    inputs,
    num_return_sequences=5,
):
    return model.generate(
        **inputs,
        temperature=0.9,
        do_sample=True,
        top_p=0.7,
        num_return_sequences=num_return_sequences,
        repetition_penalty=2.5,
        max_length=128,
    )

In [ ]:
initial_model = AutoModelForCausalLM.from_pretrained(model_path)
initial_model = initial_model.to(device)

In [ ]:
bleu = BLEUScore(n_gram=1) # n_gram=1 — пословно сравнивает
avg_bleu_score = 0.
avg_blue_score_org = 0.

cnt = 100 # на практике оценку следует прогонять по всем данным целиком
with torch.no_grad():
    for _, row in tqdm(valid["parsed"].head(cnt).items(), total=cnt):
        source = 'Первый: ' + row['turns'][0]['phrase'] + ' Второй: ' # исходная, которую подаём в модель
        target = 'Первый: ' + row['turns'][0]['phrase'] + ' Второй: ' + row['turns'][1]['phrase'] # целевая, которую ожидаем (из тестовых данных)

        inputs = tokenizer(source, return_tensors='pt') # токенизируем вход
        for key in inputs.keys():
            inputs[key] = inputs[key].to(device)

        hypotheses = generate(model, inputs, num_return_sequences=1) # generate()
        pred = tokenizer.decode(hypotheses[0], skip_special_tokens=True) # предсказание

        metric = bleu(target, pred)
        avg_bleu_score += metric.item()

        hypotheses = generate(initial_model, inputs, num_return_sequences=1) # для initial
        pred = tokenizer.decode(hypotheses[0], skip_special_tokens=True)

        metric = bleu(target, pred)
        avg_blue_score_org += metric.item()

print()
print('avg bleu:', avg_bleu_score * 1. / cnt)
print('avg initial bleu:', avg_blue_score_org * 1. / cnt)

100%|██████████| 100/100 [01:57<00:00,  1.18s/it]


avg bleu: 0.7220230600237847
avg initial bleu: 0.6273041552305222


## Расчёт метрики METEOR

In [ ]:
meteor = evaluate.load('meteor') # благодаря nltk позволяет понимать словоформы

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
avg_meteor_score = 0.
initial_meteor_score = 0.

cnt = 100
with torch.no_grad():
    for _, row in tqdm(valid["parsed"].head(cnt).items(), total=cnt):
        source = 'Первый: ' + row['turns'][0]['phrase'] + ' Второй: '
        target = 'Первый: ' + row['turns'][0]['phrase'] + ' Второй: ' + row['turns'][1]['phrase']

        inputs = tokenizer(source, return_tensors='pt')
        for key in inputs.keys():
            inputs[key] = inputs[key].to(device)

        hypotheses = generate(model, inputs, num_return_sequences=1)
        not_fine_tuned_hypotheses = generate(initial_model, inputs, num_return_sequences=1)

        pred = tokenizer.decode(hypotheses[0], skip_special_tokens=True)
        not_fine_tuned_preds = tokenizer.decode(not_fine_tuned_hypotheses[0], skip_special_tokens=True)

        metric = meteor.compute(predictions=[pred], references=[target])
        initial_metric = meteor.compute(predictions=[not_fine_tuned_preds], references=[target])

        avg_meteor_score += metric['meteor']
        initial_meteor_score += initial_metric['meteor']

print()
print('avg meteor:', avg_meteor_score * 1. / cnt)
print('avg initial meteor:', initial_meteor_score * 1. / cnt)

100%|██████████| 100/100 [02:01<00:00,  1.22s/it]


avg meteor: 0.6344132251249917
avg initial meteor: 0.4683776727421194


In [ ]:
results_dict = {}
results_dict.update({
    "Decoder_only_initial": {"METEOR": initial_meteor_score * 1. / cnt, "BLEU": avg_blue_score_org * 1. / cnt},
    "Decoder_only_tuned": {"METEOR": avg_meteor_score * 1. / cnt, "BLEU": avg_bleu_score * 1. / cnt}
})

## Итоги

In [ ]:
pd.DataFrame.from_dict(results_dict)

,Decoder_only_initial,Decoder_only_tuned
METEOR,0.468378,0.634413
BLEU,0.627304,0.722023


1. Модель дейсвительно дообучилась под задачу
2. Разрыв в метриках связан с тем, что базовая модель генерирует длинные ответы и не знает применяемый формат.